In [1]:
import pandas as pd

csv_path = '/Users/moonjeseong/Desktop/project/TeamProject1/JeseongMoon/Dataset/mulit_classification_data.csv'
df = pd.read_csv(csv_path)
print(f'{df.shape}')
df.head()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
(1941, 34)


,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,42,50,270900,270944,267,17,44,24220,76,108,...,0.8182,-0.2913,0.5822,1,0,0,0,0,0,0
1,645,651,2538079,2538108,108,10,30,11397,84,123,...,0.7931,-0.1756,0.2984,1,0,0,0,0,0,0
2,829,835,1553913,1553931,71,8,19,7972,99,125,...,0.6667,-0.1228,0.2150,1,0,0,0,0,0,0
3,853,860,369370,369415,176,13,45,18996,99,126,...,0.8444,-0.1568,0.5212,1,0,0,0,0,0,0
4,1289,1306,498078,498335,2409,60,260,246930,37,126,...,0.9338,-0.1992,1.0000,1,0,0,0,0,0,0


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
import numpy as np
import tensorflow
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
import sklearn
from sklearn.metrics import accuracy_score, log_loss

2023-05-22 00:29:40.731879: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

y = df[['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']]
X = df.iloc[:,:-7]

In [5]:
X.shape

(1941, 27)

In [6]:
X = X.rename(columns={'TypeOfSteel_A300': 'TypeOfSteel'})
X = X.drop(columns=['TypeOfSteel_A400'])
X.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas
0,42,50,270900,270944,267,17,44,24220,76,108,1687,1,80,0.0498,0.2415,0.1818,0.0047,0.4706,1.0000,1.0,2.4265,0.9031,1.6435,0.8182,-0.2913,0.5822
1,645,651,2538079,2538108,108,10,30,11397,84,123,1687,1,80,0.7647,0.3793,0.2069,0.0036,0.6000,0.9667,1.0,2.0334,0.7782,1.4624,0.7931,-0.1756,0.2984
2,829,835,1553913,1553931,71,8,19,7972,99,125,1623,1,100,0.9710,0.3426,0.3333,0.0037,0.7500,0.9474,1.0,1.8513,0.7782,1.2553,0.6667,-0.1228,0.2150
3,853,860,369370,369415,176,13,45,18996,99,126,1353,0,290,0.7287,0.4413,0.1556,0.0052,0.5385,1.0000,1.0,2.2455,0.8451,1.6532,0.8444,-0.1568,0.5212
4,1289,1306,498078,498335,2409,60,260,246930,37,126,1353,0,185,0.0695,0.4486,0.0662,0.0126,0.2833,0.9885,1.0,3.3818,1.2305,2.4099,0.9338,-0.1992,1.0000


In [7]:
X.TypeOfSteel.value_counts()

0    1164
1     777
Name: TypeOfSteel, dtype: int64

In [8]:
# 주어진 비율
ratio_0 = 1164 / (1164 + 777)
ratio_1 = 777 / (1164 + 777)

# 0과 1로 채워질 개수 계산
num_zeros = int(ratio_0 * 10000)
num_ones = int(ratio_1 * 10000)

# 0과 1로 채워진 배열 생성
expanded_binary_column = np.concatenate([np.zeros(num_zeros), np.ones(num_ones)])

# 데이터프레임에 추가하기 위해 인덱스 재설정
expanded_binary_column = pd.Series(expanded_binary_column, name='binary_column')

# 기존 데이터 X (1941, 27)
# 라벨 인코딩된 컬럼은 TypeOfSteel으로 가정하고, 0과 1로 변환
TypeOfSteel = X['TypeOfSteel'].apply(lambda x: 0 if x == 0 else 1)

# 나머지 컬럼은 기존 데이터에 노이즈를 추가하여 확장된 데이터 생성
expanded_X = np.concatenate([X, np.random.normal(0, 0.1, size=(8059, 26))], axis=0)

# 데이터프레임으로 변환
expanded_X_df = pd.DataFrame(expanded_X, columns=X.columns)

# TypeOfSteel 컬럼 교체
expanded_X_df['TypeOfSteel'] = TypeOfSteel

# binary_column 컬럼도 0과 1로 채우기
expanded_X_df['binary_column'] = expanded_binary_column

X = expanded_X_df

# 결과 출력
print(X.shape)  # (10000, 27)
print(X['binary_column'].value_counts())  # 0과 1의 개수 출력


(10000, 27)
0.0    5996
1.0    4003
Name: binary_column, dtype: int64


In [8]:
pd.set_option('display.expand_frame_repr', False) 
print(X.head())  # 데이터프레임 확인

   X_Minimum  X_Maximum  Y_Minimum  Y_Maximum  Pixels_Areas  X_Perimeter  Y_Perimeter  Sum_of_Luminosity  Minimum_of_Luminosity  Maximum_of_Luminosity  Length_of_Conveyer  TypeOfSteel  Steel_Plate_Thickness  Edges_Index  Empty_Index  Square_Index  Outside_X_Index  Edges_X_Index  Edges_Y_Index  Outside_Global_Index  LogOfAreas  Log_X_Index  Log_Y_Index  Orientation_Index  Luminosity_Index  SigmoidOfAreas
0       42.0       50.0   270900.0   270944.0         267.0         17.0         44.0            24220.0                   76.0                  108.0              1687.0            0                   80.0       0.0498       0.2415        0.1818           0.0047         0.4706         1.0000                   1.0      2.4265       0.9031       1.6435             0.8182           -0.2913          0.5822
1      645.0      651.0  2538079.0  2538108.0         108.0         10.0         30.0            11397.0                   84.0                  123.0              1687.0            1   

In [9]:
X.TypeOfSteel.value_counts()

1    5060
0    4940
Name: TypeOfSteel, dtype: int64

In [10]:
y["Type"] = y[['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']].idxmax(axis=1)
y = y.drop(columns=['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults'])
y.head()

/var/folders/jy/phdqtrzx3nb0kq5pycq5kz100000gn/T/ipykernel_11440/2028821799.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y["Type"] = y[['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']].idxmax(axis=1)


,Type
0,Pastry
1,Pastry
2,Pastry
3,Pastry
4,Pastry


In [11]:
y.Type.value_counts()

Other_Faults    673
Bumps           402
K_Scatch        391
Z_Scratch       190
Pastry          158
Stains           72
Dirtiness        55
Name: Type, dtype: int64

In [22]:
print(X.shape, y.shape)

(10000, 26) (10000,)


In [26]:
y_dataframe = df.iloc[:,-7:]

In [27]:
y_list = []
for i in range(y_dataframe.shape[0]):
    if y_dataframe["Pastry"].values[i] == 1:
        y_list.append("Pastry")
    elif y_dataframe["Z_Scratch"].values[i] == 1:
        y_list.append("Z_Scratch")
    elif y_dataframe["K_Scatch"].values[i] == 1:
        y_list.append("K_Scatch")
    elif y_dataframe["Stains"].values[i] == 1:
        y_list.append("Stains")
    elif y_dataframe["Dirtiness"].values[i] == 1:
        y_list.append("Dirtiness")
    elif y_dataframe["Bumps"].values[i] == 1:
        y_list.append("Bumps")
    else:
        y_list.append("Other_Faults")

y_list = np.array(y_list)
y_list.shape

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [25]:
label = y['Type']
print(label.unique())

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [16]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(label)
label_encoded = le.transform(label)
le_y = pd.DataFrame(label_encoded, columns=['label'])
y = le_y
print(y.value_counts())

label
3        673
0        402
2        391
6        190
4        158
5         72
1         55
dtype: int64


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 83)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape, )

ValueError: Found input variables with inconsistent numbers of samples: [10000, 1941]

In [ ]:
from sklearn import metrics
import numpy as np

def get_metrics(model, X, y):
    y_pred=model.predict(X)
    
    Precision    = metrics.precision_score (y, y_pred, average="weighted")
    Recall       = metrics.recall_score    (y, y_pred, average="weighted")
    Accuracy     = metrics.accuracy_score  (y, y_pred)
    F1           = metrics.f1_score        (y, y_pred, average="weighted")
    
    print(f'Precision Score  : {Precision}')
    print(f'Recall Score     : {Recall}')
    print(f'Accuracy Score   : {Accuracy}')
    print(f'F1 Score         : {F1}')
    return Precision, Recall, Accuracy, F1

In [ ]:
from sklearn import metrics
import numpy as np

def get_metrics(model, X, y):
    y_pred=model.predict(X)
    
    Precision    = metrics.precision_score (y, y_pred, average="weighted")
    Recall       = metrics.recall_score    (y, y_pred, average="weighted")
    Accuracy     = metrics.accuracy_score  (y, y_pred)
    F1           = metrics.f1_score        (y, y_pred, average="weighted")
    
    print(f'Precision Score  : {Precision}')
    print(f'Recall Score     : {Recall}')
    print(f'Accuracy Score   : {Accuracy}')
    print(f'F1 Score         : {F1}')
    return Precision, Recall, Accuracy, F1

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_rf_all = RandomForestClassifier(random_state=83)
model_rf_all.fit(X_train, y_train)

rf_precision_train, rf_recall_train, rf_accuracy_train, rf_f1_train = get_metrics(model_rf_all, X_train, y_train)
print('='*100)
rf_precision_test, rf_recall_test, rf_accuracy_test, rf_f1_test = get_metrics(model_rf_all, X_test, y_test)

/var/folders/jy/phdqtrzx3nb0kq5pycq5kz100000gn/T/ipykernel_11409/3915369843.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rf_all.fit(X_train, y_train)


Precision Score  : 1.0
Recall Score     : 1.0
Accuracy Score   : 1.0
F1 Score         : 1.0
Precision Score  : 0.7992110315751305
Recall Score     : 0.7866323907455013
Accuracy Score   : 0.7866323907455013
F1 Score         : 0.7892523019803969


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from sklearn.metrics import accuracy_score, log_loss

In [ ]:
from sklearn.utils import class_weight

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='LeakyReLU', input_shape=(X_train.shape[1],)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),  # 드롭아웃 추가
    tf.keras.layers.Dense(256, activation='LeakyReLU', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01)),  # L1, L2 정규화 적용
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),  # 드롭아웃 추가
    tf.keras.layers.Dense(128, activation='LeakyReLU', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01)),  # L1, L2 정규화 적용
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),  # 드롭아웃 추가
    tf.keras.layers.Dense(64, activation='LeakyReLU', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01)),  # L1, L2 정규화 적용
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),  # 드롭아웃 추가
    tf.keras.layers.Dense(32, activation='LeakyReLU', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01)),  # L1, L2 정규화 적용
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),  # 드롭아웃 추가
    tf.keras.layers.Dense(16, activation='LeakyReLU', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01)),  # L1, L2 정규화 적용
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),  # 드롭아웃 추가
    tf.keras.layers.Dense(7, activation='softmax')
])


optimizer = tf.keras.optimizers.Adamax()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 학습 자동 중단 설정
rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=40, mode='min')

# 모델 학습
model.fit(X_train, tf.keras.utils.to_categorical(y_train), epochs=100, batch_size=10, verbose=1, 
          validation_data=(X_test, tf.keras.utils.to_categorical(y_test)), callbacks=[rlrp])

# 모델 평가
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # 클래스 예측값으로 변환
accuracy = accuracy_score(y_test, y_pred_classes)
loss = log_loss(y_test, y_pred)
print(f'Accuracy: {accuracy}\nLog Loss: {loss}')

Epoch 1/100
156/156 [==============================] - 3s 5ms/step - loss: 80.8849 - accuracy: 0.2983 - val_loss: 68.9232 - val_accuracy: 0.3650 - lr: 0.0010
Epoch 2/100
156/156 [==============================] - 0s 3ms/step - loss: 61.9869 - accuracy: 0.4014 - val_loss: 55.5550 - val_accuracy: 0.3907 - lr: 0.0010
Epoch 3/100
156/156 [==============================] - 0s 3ms/step - loss: 50.4436 - accuracy: 0.3950 - val_loss: 45.5578 - val_accuracy: 0.3779 - lr: 0.0010
Epoch 4/100
156/156 [==============================] - 0s 3ms/step - loss: 41.6348 - accuracy: 0.4117 - val_loss: 37.8464 - val_accuracy: 0.2879 - lr: 0.0010
Epoch 5/100
156/156 [==============================] - 0s 3ms/step - loss: 34.7139 - accuracy: 0.4175 - val_loss: 31.7385 - val_accuracy: 0.4036 - lr: 0.0010
Epoch 6/100
156/156 [==============================] - 0s 3ms/step - loss: 29.1290 - accuracy: 0.4394 - val_loss: 26.6634 - val_accuracy: 0.4267 - lr: 0.0010
Epoch 7/100
156/156 [==============================]

KeyboardInterrupt: 